<a href="https://colab.research.google.com/github/sungjk1999/smart-factory-quality-classification/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Read train data**

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from google.colab import drive
drive.mount('/content/gdrive')


"""
PRODUCT_ID : 제품의 고유 ID

Y_Class : 제품 품질 상태(Target) 
    0 : 적정 기준 미달 (부적합) 
    1 : 적합 
    2 : 적정 기준 초과 (부적합)

Y_Quality : 제품 품질 관련 정량적 수치 

TIMESTAMP : 제품이 공정에 들어간 시각 

LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재) 

PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재) 

X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수
"""


dir_path = "/content/gdrive/MyDrive/Colab Notebooks/open"

train = pd.read_csv(f"{dir_path}/train.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Remove invalid columns**


In [59]:
# X_1 ~ X_2875 열의 std가 0 또는 nan인 경우를 제거
# test.csv 에서도 동일하게 적용해야함.

invalid = []

tmp = train.describe()
for col in tmp.columns:
    std = tmp.loc['std', col]
    if std == 0 or np.isnan(std):
        invalid.append(col)

# 아래 두 변수가 실제로 의미가 없는지 판단
PRINT = 0

if PRINT:
    plt.figure(figsize=(12, 5))

    # PRODUCT_ID
    ax = plt.subplot(1, 2, 1)
    ax.axes.xaxis.set_ticks([])
    plt.xlabel("PRODUCT_ID")
    plt.scatter(train["PRODUCT_ID"], y=train["Y_Quality"], c='red', alpha=0.2)

    # TIMESTAMP
    ax = plt.subplot(1, 2, 2)
    ax.axes.xaxis.set_ticks([])
    tmp = train.sort_values('TIMESTAMP')
    plt.xlabel("TIMESTAMP")
    plt.scatter(tmp["TIMESTAMP"], y=tmp["Y_Quality"], c='green', alpha=0.2)

# 실제로 무관하므로 제거
invalid += ["PRODUCT_ID", "TIMESTAMP"]

train = train.drop(columns=invalid)
train

,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,1,0.533433,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77
1,2,0.541819,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55
2,1,0.531267,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35
3,2,0.537325,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78
4,1,0.531590,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,1,0.526546,T100306,T_31,2.0,95.0,10.0,50.0,10.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,0,0.524022,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,180.810345,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49
595,0,0.521289,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,176.486207,156.6,383.0,367.018868,352.0,NaN,NaN,NaN,NaN,NaN
596,1,0.531375,T100304,O_31,40.0,94.0,11.0,45.0,10.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **How to classify?**



In [60]:
# Y_Quality 값을 통해서, Y_Class를 예측하므로 어느 정도에서 끊기는지 확인
# 정량 데이터이므로, 구간이 겹치지 않는 것을 확인할 수 있음.

over = train[train['Y_Class'] == 2]['Y_Quality']
normal = train[train['Y_Class'] == 1]['Y_Quality']
under = train[train['Y_Class'] == 0]['Y_Quality']

print(f"적정 기준 초과 여부 커트라인 구간: [{normal.max()}, {over.min()}]")
print(f"적정 기준 미달 여부 커트라인 구간: [{under.max()}, {normal.min()}]")

적정 기준 초과 여부 커트라인 구간: [0.534842857, 0.534950794]
적정 기준 미달 여부 커트라인 구간: [0.525066667, 0.525085714]


In [61]:
# 커트라인에 따른 분류 함수1 - 확정적인 경우

def classify(train, y_quality):
    pass

In [62]:
# LINE에 따라서 어떻게 분포하는지 확인